# Multiprocessing on shared read-only structure

## Without OO

In [6]:
# Source: https://stackoverflow.com/a/60348772/4563947
# source: https://stackoverflow.com/questions/5549190/is-shared-readonly-data-copied-to-different-processes-for-multiprocessing
import multiprocessing
from time import sleep


if __name__ == "__main__":
    my_global = []
  
    def my_func(i, d=my_global):
        print(hex(id(d)))
        sleep(d[i])
        print(d[i])
        return d[i]
    
    print(f'main proc: {my_global} at {hex(id(my_global))}')
    my_global.extend([pow(i, 0.5) for i in range(10)])
    print(f'main proc: {my_global} at {hex(id(my_global))}')
    
    pool = multiprocessing.Pool(processes=4)
    i = pool.map(my_func, range(10))
    
    print("i = ",i)
    

main proc: [] at 0x7ff00433e480
main proc: [0.0, 1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 2.23606797749979, 2.449489742783178, 2.6457513110645907, 2.8284271247461903, 3.0] at 0x7ff00433e480
0x7ff00433e4800x7ff00433e4800x7ff00433e480
0x7ff00433e480


0.0
0x7ff00433e480
1.0
0x7ff00433e480
1.4142135623730951
0x7ff00433e480
1.7320508075688772
0x7ff00433e480
2.0
0x7ff00433e480
2.23606797749979
0x7ff00433e480
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
i =  [0.0, 1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 2.23606797749979, 2.449489742783178, 2.6457513110645907, 2.8284271247461903, 3.0]


## With OO

In [1]:
# Source: https://stackoverflow.com/a/60348772/4563947
# source: https://stackoverflow.com/questions/5549190/is-shared-readonly-data-copied-to-different-processes-for-multiprocessing
# source: https://stackoverflow.com/questions/29009790/python-how-to-do-multiprocessing-inside-of-a-class
import multiprocessing
from time import sleep

class Reference:
    def __init__(self):
        self.reference = None


class Printer:
    l = None
    def __init__(self, l):
        Printer.l = l
        
    def multiprocess(self):
        
        pool = multiprocessing.Pool(processes=10)
        i = pool.map(self.process, range(10))
        
        print("i = ",i)
        
    def process(self, i):
        data = Printer.l
        print(hex(id(data)))
        j = 0
        while(data[i] * 10000000 > j):
            pow(i, 0.5)
            j+=1
        print(data[i])
        ref = Reference()
        ref.reference = data[i]
        return ref


if __name__ == "__main__":
    my_global = []
    
    print(f'main proc: {my_global} at {hex(id(my_global))}')
    my_global.extend([pow(i, 0.5) for i in range(10)])
    ref1 = Reference()
    ref2 = Reference()
    ref1.reference = ref2
    ref2.reference = ref1
    # this can not be pickled
    my_global.append(ref1)
    print(f'main proc: {my_global} at {hex(id(my_global))}')
    
    p = Printer(my_global)
    p.multiprocess()
    

main proc: [] at 0x7f2184671800
main proc: [0.0, 1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 2.23606797749979, 2.449489742783178, 2.6457513110645907, 2.8284271247461903, 3.0, <__main__.Reference object at 0x7f2184676280>] at 0x7f2184671800
0x7f21846718000x7f21846718000x7f21846718000x7f21846718000x7f2184671800
0x7f21846718000x7f21846718000x7f2184671800
0x7f2184671800

0.0

0x7f2184671800




1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
i =  [<__main__.Reference object at 0x7f21845a5040>, <__main__.Reference object at 0x7f218459d760>, <__main__.Reference object at 0x7f21845a5190>, <__main__.Reference object at 0x7f218459dfd0>, <__main__.Reference object at 0x7f218459dfa0>, <__main__.Reference object at 0x7f21845a52e0>, <__main__.Reference object at 0x7f21845a52b0>, <__main__.Reference object at 0x7f21845a5100>, <__main__.Reference object at 0x7f21845a5280>, <__main__.Reference object at 0x7f21845a5520>]


In [3]:
# source: https://stackoverflow.com/questions/41920124/multiprocessing-use-tqdm-to-display-a-progress-bar
from multiprocessing import Pool
import time
from tqdm import *

def _foo(my_number):
   square = my_number * my_number
   time.sleep(1)
   return square 

if __name__ == '__main__':
    with Pool(processes=2) as p:
        max_ = 30
        with tqdm(total=max_) as pbar:
            for i, _ in enumerate(p.imap_unordered(_foo, range(0, max_))):
                pbar.update()

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:15<00:00,  2.00it/s]


In [4]:
# source: https://github.com/tqdm/tqdm/issues/407
# source: https://github.com/tqdm/tqdm#nested-progress-bars
from time import sleep
from tqdm import tqdm
from multiprocessing import Pool, freeze_support

def progresser(n):
    text = "progresser #{}".format(n)
    for i in tqdm(range(5000), desc=text, position=n):
        sleep(0.001)

if __name__ == '__main__':
    freeze_support()  # for Windows support
    L = list(range(10))
    Pool(len(L)).map(progresser, L)

progresser #1:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]

progresser #2:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]


progresser #3:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]




progresser #5:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]







progresser #8:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]



progresser #4:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]





progresser #6:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]






progresser #7:   0%|                                                                        | 0/5000 [00:00<?, ?it/s]








progresser #1:   2%|